In [1]:
from ipywidgets import HTML
import ipysheet
from ipyleaflet import Map, Marker, Popup,Icon
from credentials import uri
import pymongo
import pandas as pd   
import ipywidgets as widgets

red_icon_url = "http://maps.google.com/mapfiles/kml/pushpin/red-pushpin.png"
green_icon_url ="http://maps.google.com/mapfiles/kml/pushpin/grn-pushpin.png"
blue_icon_url = "http://maps.google.com/mapfiles/kml/pushpin/blue-pushpin.png"

In [2]:
def init_db():
    client = pymongo.MongoClient(uri)
    DB_NAME = "test-db"
    db = client[DB_NAME]
    return db

In [3]:
def getData():
    db = init_db()
    collection = db.plugsurfing_cologne
    data = collection.find({})
    data = list(data)
    df1 = pd.DataFrame(data)   

    collection = db.plugshare_cologne
    data = collection.find({})
    data = list(data)
    df2 = pd.DataFrame(data)   

    collection = db.placetoplug_cologne
    data = collection.find({})
    data = list(data)
    df3 = pd.DataFrame(data)   
    return df1, df2, df3


In [4]:
from ipysheet import from_array

def display_data(map):
    df1,df2,df3 = getData()
    offset = 0.0001

    #plugsurfing
    for _, row in df1.iterrows():
        icon = Icon(icon_url=blue_icon_url, icon_size=[35, 35],draggable=False)
        marker = Marker(location=(row.latitude, row.longitude), icon= icon)
        #marker.popup(ipysheet.from_dataframe(row.to_frame()))
        sheet = HTML()
        sheet.value = row.to_string()
        marker.popup = sheet
        map.add_layer(marker)

    #plugshare
    for _, row in df2.iterrows():
        icon = Icon(icon_url=green_icon_url, icon_size=[35, 35],draggable=False)
        marker = Marker(location=(row.latitude +offset, row.longitude +offset), icon= icon)
        #data = row.to_numpy()
        #print(data)
        #print(type(data))
        #sheet = from_array(data)
        sheet = HTML()
        sheet.value = row.to_string()
        marker.popup = sheet
        map.add_layer(marker)

    #place to plug
    for _, row in df3.iterrows():
        icon = Icon(icon_url=red_icon_url, icon_size=[35, 35],draggable=False)
        marker = Marker(location=(row.coordinates[1] - offset, row.coordinates[0] - offset), icon= icon)
        #marker.popup(ipysheet.from_dataframe(row.to_frame()))
        sheet = HTML()
        sheet.value = row.to_string()
        marker.popup = sheet
        map.add_layer(marker)
    return map

In [5]:
def createMap():
    center = (50.94151,6.955385)
    m = Map(center=center, zoom=12, close_popup_on_click=True)
    display_data(m)
    return m


In [7]:
# green - plugshare (closed API)
# blue - plugsurfing (opensoruce 2017)
# red - placetoplug (limit auf 50)
m = createMap()
m

Map(center=[50.94151, 6.955385], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…